# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [9]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [11]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [12]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [26]:
data_total = data.groupby(["CustomerID", "ProductName"]).agg({"Quantity": "sum"})

df = pd.DataFrame(data=data_total)
df

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [32]:
table = pd.pivot_table(df, values='Quantity', index=['ProductName'], columns=['CustomerID'], fill_value=0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [48]:
distances = pdist(table.T, "euclidean")
Q = squareform(distances)

In [49]:
pd.DataFrame(1/(1+Q), index=table.columns,columns=table.columns)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [55]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)


similarities = distances[33].sort_values(ascending=False)[1:6]
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

In [81]:
df_sim = pd.DataFrame(data=similarities)
df_sim

,33
CustomerID,
264,0.087047
3535,0.087047
3317,0.087047
2503,0.085983
3305,0.085638


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [86]:
records = df.loc[df_sim.index]
records

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [93]:
cool_records = records.groupby("ProductName").agg({"Quantity": "sum"}).sort_values(by="Quantity", ascending=False)
cool_records

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [104]:
coolest = cool_records.merge(table, left_on="ProductName", right_on="ProductName")
coolest

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Butter - Unsalted,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
Wine - Ej Gallo Sierra Valley,3,0,0,0,1,0,0,0,0,0,...,0,0,25,50,0,25,0,0,0,0
Towels - Paper / Kraft,3,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Soup - Campbells Bean Medley,3,0,0,1,0,0,0,2,0,0,...,0,0,0,25,50,25,0,0,0,0
Wine - Blue Nun Qualitatswein,3,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hinge W Undercut,1,0,0,1,0,0,0,0,0,1,...,25,0,0,25,25,0,0,0,0,0
Ice Cream Bar - Hageen Daz To,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,25,0,0,0
Jagermeister,1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,25,0


In [136]:
df33 = coolest[["Quantity", 33]]
df33[df33[33]==0][:5].index.tolist()   # in order to change the df to list format

['Knife Plastic - White',
 'Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Wine - Blue Nun Qualitatswein',
 'Beef Wellington']

**We would recommend => Butter, Wine, Soup, Wine, Chicken**

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [125]:
empty_dict = {}

cust_list = data["CustomerID"].unique()

In [132]:
for i in cust_list:

    similarities = distances[i].sort_values(ascending=False)[1:6]
    
    df_sim = pd.DataFrame(data=similarities)
    
    records = df.loc[df_sim.index]
    
    cool_records = records.groupby("ProductName").agg({"Quantity": "sum"}).sort_values(by="Quantity", ascending=False)
    
    coolest = cool_records.merge(table, left_on="ProductName", right_on="ProductName")
    
    df_all = coolest[["Quantity", i]]
    df_all_clean = df_all[df_all[i]==0]
    
    final = df_all_clean[:5].index.tolist()
    empty_dict[i] = final

In [133]:
empty_dict

{61288: ['Jagermeister',
  'Chicken - Soup Base',
  'Lime Cordial - Roses',
  'Macaroons - Two Bite Choc',
  'Flavouring - Orange'],
 77352: ['Wine - Valpolicella Masi',
  'Tahini Paste',
  'Tuna - Salad Premix',
  'Chicken - Soup Base',
  'Isomalt'],
 40094: ['Bread - Italian Corn Meal Poly',
  'Puree - Mocha',
  'Tuna - Salad Premix',
  'Sherry - Dry',
  'Beer - Sleemans Cream Ale'],
 23548: ['Squid - Tubes / Tenticles 10/20',
  'Sprouts - Baby Pea Tendrils',
  'Flavouring - Orange',
  'Bread - Raisin Walnut Oval',
  'Banana Turning'],
 78981: ['Vanilla Beans',
  'Lettuce - Frisee',
  'Yogurt - Blueberry, 175 Gr',
  'Cinnamon Buns Sticky',
  'Pop Shoppe Cream Soda'],
 83106: ['Cocoa Butter',
  'Cheese - Boursin, Garlic / Herbs',
  'Garlic - Peeled',
  'Cheese - Mix',
  'Sauce - Hollandaise'],
 11253: ['Juice - Lime',
  'Tomatoes Tear Drop',
  'Pomello',
  'Potatoes - Instant, Mashed',
  'Table Cloth - 53x69 Colour'],
 35107: ['Bread - French Baquette',
  'Bandage - Flexible Neon',
  

---------

##  Step 9: Store the results in a Pandas data frame. The data frame should have a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [139]:
all_reco = pd.DataFrame(data=empty_dict)
all_reco

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,10114,25270,17390,97029,38154,74403,49005,41286,85878,68506
0,Jagermeister,Wine - Valpolicella Masi,Bread - Italian Corn Meal Poly,Squid - Tubes / Tenticles 10/20,Vanilla Beans,Cocoa Butter,Juice - Lime,Bread - French Baquette,Soup Knorr Chili With Beans,Cheese - Mozzarella,...,Turkey - Oven Roast Breast,Beef - Top Sirloin,"Pasta - Penne, Rigate, Dry",Milk - 2%,Cheese - Mix,Chips Potato Salt Vinegar 43g,"Pasta - Penne, Rigate, Dry",Veal - Inside,Olives - Kalamata,"Veal - Brisket, Provimi,bnls"
1,Chicken - Soup Base,Tahini Paste,Puree - Mocha,Sprouts - Baby Pea Tendrils,Lettuce - Frisee,"Cheese - Boursin, Garlic / Herbs",Tomatoes Tear Drop,Bandage - Flexible Neon,Soupfoamcont12oz 112con,Bananas,...,"Tart Shells - Sweet, 4",Grouper - Fresh,Coffee - Hazelnut Cream,Bay Leaf,Rum - Mount Gay Eclipes,Fenngreek Seed,Browning Caramel Glace,"Thyme - Lemon, Fresh",Wine - White Cab Sauv.on,Beef - Ground Medium
2,Lime Cordial - Roses,Tuna - Salad Premix,Tuna - Salad Premix,Flavouring - Orange,"Yogurt - Blueberry, 175 Gr",Garlic - Peeled,Pomello,"Lentils - Red, Dry",Veal - Sweetbread,Quiche Assorted,...,Wine - Blue Nun Qualitatswein,Soup Knorr Chili With Beans,Soup Knorr Chili With Beans,"Veal - Inside, Choice",Muffin Mix - Blueberry,Isomalt,Tea - Decaf Lipton,Wonton Wrappers,Veal - Eye Of Round,"Rosemary - Primerba, Paste"
3,Macaroons - Two Bite Choc,Chicken - Soup Base,Sherry - Dry,Bread - Raisin Walnut Oval,Cinnamon Buns Sticky,Cheese - Mix,"Potatoes - Instant, Mashed",Thermometer Digital,Cocktail Napkin Blue,"Lemonade - Natural, 591 Ml",...,Crackers - Trio,Eggplant - Asian,Sardines,Banana Turning,Cake - Box Window 10x10x2.5,Bay Leaf,"Garlic - Primerba, Paste","Veal - Inside, Choice",Bread - Calabrese Baguette,Scallops - 10/20
4,Flavouring - Orange,Isomalt,Beer - Sleemans Cream Ale,Banana Turning,Pop Shoppe Cream Soda,Sauce - Hollandaise,Table Cloth - 53x69 Colour,Wine - Hardys Bankside Shiraz,Bread - Raisin Walnut Oval,Coffee - Hazelnut Cream,...,Otomegusa Dashi Konbu,Lime Cordial - Roses,"Chicken - Leg, Boneless",Wine - Vidal Icewine Magnotta,Scallops 60/80 Iqf,Veal - Sweetbread,"Wine - Red, Harrow Estates, Cab",Bread Crumbs - Japanese Style,Cheese - Wine,"Oregano - Dry, Rubbed"


In [ ]:
all_reco = pd.DataFrame(data=empty_dict, columns=["CustomerID", "ProductNAme"])
all_reco

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [140]:
distances10 = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'jaccard'))), 
                         index=table.columns, columns=table.columns)

In [142]:
empty_dict10 = {}

cust_list10 = data["CustomerID"].unique()

In [145]:
for i in cust_list:

    similarities = distances10[i].sort_values(ascending=False)[1:6]
    
    df_sim = pd.DataFrame(data=similarities)
    
    records = df.loc[df_sim.index]
    
    cool_records = records.groupby("ProductName").agg({"Quantity": "sum"}).sort_values(by="Quantity", ascending=False)
    
    coolest = cool_records.merge(table, left_on="ProductName", right_on="ProductName")
    
    df_all = coolest[["Quantity", i]]
    df_all_clean = df_all[df_all[i]==0]
    
    final = df_all_clean[:5].index.tolist()
    empty_dict10[i] = final

In [146]:
empty_dict10

{61288: ['Cheese - Brie, Triple Creme',
  'V8 - Berry Blend',
  'Ecolab - Mikroklene 4/4 L',
  'Flavouring - Orange',
  'Mustard - Seed'],
 77352: ['Cheese - Wine',
  'Sea Bass - Whole',
  'Garlic',
  'Beef - Top Sirloin - Aaa',
  'Beer - Blue'],
 40094: ['Juice - Cranberry, 341 Ml',
  'Rosemary - Dry',
  'Wine - Red, Colio Cabernet',
  'Sponge Cake Mix - Chocolate',
  'Pasta - Angel Hair'],
 23548: ['Wine - Hardys Bankside Shiraz',
  'Table Cloth 81x81 White',
  'Cookies Cereal Nut',
  'Longos - Chicken Wings',
  'Wine - Cahors Ac 2000, Clos'],
 78981: ['Muffin Batt - Blueberry Passion',
  'Pie Filling - Cherry',
  'Cocoa Butter',
  'Wine - Blue Nun Qualitatswein',
  'Cheese - Cottage Cheese'],
 83106: ['Grouper - Fresh',
  'Salmon - Sockeye Raw',
  'Tea - Earl Grey',
  'Pork - Inside',
  'Mustard Prepared'],
 11253: ['Ocean Spray - Kiwi Strawberry',
  'Wine - Magnotta, Merlot Sr Vqa',
  'Quiche Assorted',
  'Cheese - Victor Et Berthold',
  'Veal - Osso Bucco'],
 35107: ['Placemat - S

-------

In [147]:
all_reco10 = pd.DataFrame(data=empty_dict10)
all_reco10

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,"Cheese - Brie, Triple Creme",Cheese - Wine,"Juice - Cranberry, 341 Ml",Wine - Hardys Bankside Shiraz,Muffin Batt - Blueberry Passion,Grouper - Fresh,Ocean Spray - Kiwi Strawberry,"Placemat - Scallop, White",Zucchini - Yellow,Muffin - Carrot Individual Wrap,...,Pie Filling - Cherry,French Pastry - Mini Chocolate,Onion Powder,"Pork - Back, Short Cut, Boneless",Tahini Paste,Juice - Happy Planet,Wine - Redchard Merritt,Scampi Tail,"Cup - 6oz, Foam",Cumin - Whole
1,V8 - Berry Blend,Sea Bass - Whole,Rosemary - Dry,Table Cloth 81x81 White,Pie Filling - Cherry,Salmon - Sockeye Raw,"Wine - Magnotta, Merlot Sr Vqa",Crackers - Trio,Kellogs Special K Cereal,Remy Red,...,Rum - Mount Gay Eclipes,Bacardi Breezer - Tropical,Scampi Tail,"Oregano - Dry, Rubbed",Pastry - Raisin Muffin - Mini,Lettuce - Frisee,Towels - Paper / Kraft,Lettuce - Treviso,"Coconut - Shredded, Sweet","Beef - Tenderlion, Center Cut"
2,Ecolab - Mikroklene 4/4 L,Garlic,"Wine - Red, Colio Cabernet",Cookies Cereal Nut,Cocoa Butter,Tea - Earl Grey,Quiche Assorted,Mustard Prepared,Cinnamon Buns Sticky,Sword Pick Asst,...,Lettuce - Treviso,Juice - Lime,Bouq All Italian - Primerba,"Beef - Chuck, Boneless",Tea - Herbal Sweet Dreams,Eggplant - Asian,Hot Chocolate - Individual,Zucchini - Yellow,"Wine - Cahors Ac 2000, Clos",Sprouts - Baby Pea Tendrils
3,Flavouring - Orange,Beef - Top Sirloin - Aaa,Sponge Cake Mix - Chocolate,Longos - Chicken Wings,Wine - Blue Nun Qualitatswein,Pork - Inside,Cheese - Victor Et Berthold,Broom - Corn,Beef Wellington,Wine - Hardys Bankside Shiraz,...,Cup - Translucent 7 Oz Clear,Cornflakes,Tea - Jasmin Green,Cheese - Cheddarsliced,Oil - Shortening - All - Purpose,Smoked Paprika,Berry Brulee,Bread Fig And Almond,Rice - Jasmine Sented,Spice - Peppercorn Melange
4,Mustard - Seed,Beer - Blue,Pasta - Angel Hair,"Wine - Cahors Ac 2000, Clos",Cheese - Cottage Cheese,Mustard Prepared,Veal - Osso Bucco,Sausage - Breakfast,Pie Filling - Cherry,Cheese - Mix,...,Cheese Cloth No 100,Cookies Cereal Nut,Tomato - Tricolor Cherry,Ketchup - Tomato,Dc Hikiage Hira Huba,Mussels - Frozen,"Lamb - Pieces, Diced",Pastry - Cheese Baked Scones,"Soup - Canadian Pea, Dry Mix",Curry Paste - Madras
